In [1]:
using DataFrames
using Statistics
using Dates
using XLSX

In [3]:
λ = 0.70931715
ΔtthD = 0.005
px = 0.1353
standards = Dict(
    "Si_small" => 5.431042,
    "Ge" => 5.65789,
    "LaB6" => 4.15692,
    "BGO_40" => 10.5182,
)

Dict{String, Float64} with 4 entries:
  "LaB6"     => 4.15692
  "BGO_40"   => 10.5182
  "Ge"       => 5.65789
  "Si_small" => 5.43104

In [12]:
function bond_calc(h, k, l, D, tthD, xp, xn)
    global λ, ΔtthD, px, a0
    tth0 = 2asind(λ * hypot(h, k, l) / 2a0)
    tth = tthD - rad2deg(px / D) * (xp - xn) / 2
    a = hypot(h, k, l) * λ / 2sind(tth / 2)
    tth, tth - tth0, a, a - a0
end

bond_calc (generic function with 1 method)

In [5]:
function hklfam(h, k, l)
    Tuple(sort(abs.([h, k, l]), rev=true))
end

hklfam (generic function with 1 method)

In [6]:
function hklnorm2(h, k, l)
    h^2 + k^2 + l^2
end

hklnorm2 (generic function with 1 method)

In [7]:
function hklpos(h, k, l)
    h != 0 ? h > 0 :
    k != 0 ? k > 0 :
    l > 0
end

hklpos (generic function with 1 method)

In [13]:
name = "BGO_40"
a0 = standards[name]
data = DataFrame(XLSX.readtable("$(name).xlsx", name))

Row,nframe,frame,norient,orient,created,distance,tthD,omega,phi,inc,time,h,k,l,x1,y1,x2,y2,Imax,background,A1,σx1,σy1,A2,σx2,σy2
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,1,mo_BGO40_17_14_3_Family_129f_Bond_MP97_02_0001.sfrm,2,20240718_BGO_40.p4p,2024-07-19T08:40:28,128.5,-97.0,-90.6,34.7796,4.0,1800.0,-14,-17,3,389.91,504.614,402.953,504.69,4574,406.897,4222.58,1.56374,1.4083,2254.61,1.67666,1.49195
2,2,mo_BGO40_17_14_3_Family_129f_Bond_MP97_05_0001.sfrm,2,20240718_BGO_40.p4p,2024-07-19T09:11:30,128.5,-97.0,89.4,34.7796,4.0,1800.0,14,17,-3,390.3,503.748,403.448,503.903,4329,442.888,4232.83,1.6236,1.54672,2276.27,1.68874,1.52489
3,3,mo_BGO40_17_14_3_Family_129f_Bond_MP97_08_0001.sfrm,2,20240718_BGO_40.p4p,2024-07-19T09:42:52,128.5,97.0,86.3,36.1206,4.0,1800.0,14,-3,17,387.52,512.709,374.659,513.208,4223,552.633,3372.25,1.80437,2.68362,1916.52,1.90198,2.47793
4,4,mo_BGO40_17_14_3_Family_129f_Bond_MP97_11_0001.sfrm,2,20240718_BGO_40.p4p,2024-07-19T10:13:53,128.5,97.0,-93.7,36.1206,4.0,600.0,-14,3,-17,387.621,492.175,374.733,492.525,2095,178.753,2047.18,1.77131,1.49563,1023.55,1.88262,1.57206


In [14]:
bond = innerjoin(data[data.tthD .> 0, :], transform(data, :tthD => -, renamecols = false),
    on = [:tthD, :distance, :phi, :h, :k, :l, :norient, :orient],
    renamecols = "_pos" => "_neg",
)
subset!(bond, [:created_pos, :created_neg] => ByRow((c1, c2) -> abs(c1 - c2) < Day(1)))
transform!(bond, [:h, :k, :l, :distance, :tthD, :x1_pos, :x1_neg] => ByRow(bond_calc) => [:tth, :Δtth, :a, :Δa])

Row,nframe_pos,frame_pos,norient,orient,created_pos,distance,tthD,omega_pos,phi,inc_pos,time_pos,h,k,l,x1_pos,y1_pos,x2_pos,y2_pos,Imax_pos,background_pos,A1_pos,σx1_pos,σy1_pos,A2_pos,σx2_pos,σy2_pos,nframe_neg,frame_neg,created_neg,omega_neg,inc_neg,time_neg,x1_neg,y1_neg,x2_neg,y2_neg,Imax_neg,background_neg,A1_neg,σx1_neg,σy1_neg,A2_neg,σx2_neg,σy2_neg,tth,Δtth,a,Δa
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any


In [15]:
frid = innerjoin(subset(data, [:h, :k, :l] => ByRow(hklpos)), transform(data, [:h, :k, :l] .=> -, renamecols = false),
    on = [:tthD, :distance, :phi, :h, :k, :l, :norient, :orient],
    renamecols = "" => "_frid"
)
subset!(frid, [:created, :created_frid] => ByRow((c1, c2) -> abs(c1 - c2) < Day(1)))
transform!(frid, [:x1, :x1_frid] => ByRow((x, xf) -> (x + xf) / 2) => :x1_mean)
transform!(frid, [:h, :k, :l] => ByRow(hklnorm2) => :hklnorm2)

Row,nframe,frame,norient,orient,created,distance,tthD,omega,phi,inc,time,h,k,l,x1,y1,x2,y2,Imax,background,A1,σx1,σy1,A2,σx2,σy2,nframe_frid,frame_frid,created_frid,omega_frid,inc_frid,time_frid,x1_frid,y1_frid,x2_frid,y2_frid,Imax_frid,background_frid,A1_frid,σx1_frid,σy1_frid,A2_frid,σx2_frid,σy2_frid,x1_mean,hklnorm2
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Float64,Int64
1,2,mo_BGO40_17_14_3_Family_129f_Bond_MP97_05_0001.sfrm,2,20240718_BGO_40.p4p,2024-07-19T09:11:30,128.5,-97.0,89.4,34.7796,4.0,1800.0,14,17,-3,390.3,503.748,403.448,503.903,4329,442.888,4232.83,1.6236,1.54672,2276.27,1.68874,1.52489,1,mo_BGO40_17_14_3_Family_129f_Bond_MP97_02_0001.sfrm,2024-07-19T08:40:28,-90.6,4.0,1800.0,389.91,504.614,402.953,504.69,4574,406.897,4222.58,1.56374,1.4083,2254.61,1.67666,1.49195,390.105,494
2,3,mo_BGO40_17_14_3_Family_129f_Bond_MP97_08_0001.sfrm,2,20240718_BGO_40.p4p,2024-07-19T09:42:52,128.5,97.0,86.3,36.1206,4.0,1800.0,14,-3,17,387.52,512.709,374.659,513.208,4223,552.633,3372.25,1.80437,2.68362,1916.52,1.90198,2.47793,4,mo_BGO40_17_14_3_Family_129f_Bond_MP97_11_0001.sfrm,2024-07-19T10:13:53,-93.7,4.0,600.0,387.621,492.175,374.733,492.525,2095,178.753,2047.18,1.77131,1.49563,1023.55,1.88262,1.57206,387.571,494


In [16]:
eccentr = innerjoin(frid[frid.tthD .> 0, :], transform(frid, :tthD => -, renamecols = false),
    on = [:tthD, :distance, :hklnorm2, :norient, :orient],
    renamecols = "_pos" => "_neg"
)
subset!(eccentr, [:created_pos, :created_neg] => ByRow((c1, c2) -> abs(c1 - c2) < Day(1)))
subset!(eccentr, [:phi_pos, :phi_neg] => ByRow((p1, p2) -> abs(rem(p1 - p2, 360, RoundNearest)) < 15))
transform!(eccentr, [:h_pos, :k_pos, :l_pos, :distance, :tthD, :x1_mean_pos, :x1_mean_neg] => ByRow(bond_calc) => [:tth, :Δtth, :a, :Δa])

Row,nframe_pos,frame_pos,norient,orient,created_pos,distance,tthD,omega_pos,phi_pos,inc_pos,time_pos,h_pos,k_pos,l_pos,x1_pos,y1_pos,x2_pos,y2_pos,Imax_pos,background_pos,A1_pos,σx1_pos,σy1_pos,A2_pos,σx2_pos,σy2_pos,nframe_frid_pos,frame_frid_pos,created_frid_pos,omega_frid_pos,inc_frid_pos,time_frid_pos,x1_frid_pos,y1_frid_pos,x2_frid_pos,y2_frid_pos,Imax_frid_pos,background_frid_pos,A1_frid_pos,σx1_frid_pos,σy1_frid_pos,A2_frid_pos,σx2_frid_pos,σy2_frid_pos,x1_mean_pos,hklnorm2,nframe_neg,frame_neg,created_neg,omega_neg,phi_neg,inc_neg,time_neg,h_neg,k_neg,l_neg,x1_neg,y1_neg,x2_neg,y2_neg,Imax_neg,background_neg,A1_neg,σx1_neg,σy1_neg,A2_neg,σx2_neg,σy2_neg,nframe_frid_neg,frame_frid_neg,created_frid_neg,omega_frid_neg,inc_frid_neg,time_frid_neg,x1_frid_neg,y1_frid_neg,x2_frid_neg,y2_frid_neg,Imax_frid_neg,background_frid_neg,A1_frid_neg,σx1_frid_neg,σy1_frid_neg,A2_frid_neg,σx2_frid_neg,σy2_frid_neg,x1_mean_neg,tth,Δtth,a,Δa
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Float64,Int64,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Float64,Float64,Float64,Float64,Float64
1,3,mo_BGO40_17_14_3_Family_129f_Bond_MP97_08_0001.sfrm,2,20240718_BGO_40.p4p,2024-07-19T09:42:52,128.5,97.0,86.3,36.1206,4.0,1800.0,14,-3,17,387.52,512.709,374.659,513.208,4223,552.633,3372.25,1.80437,2.68362,1916.52,1.90198,2.47793,4,mo_BGO40_17_14_3_Family_129f_Bond_MP97_11_0001.sfrm,2024-07-19T10:13:53,-93.7,4.0,600.0,387.621,492.175,374.733,492.525,2095,178.753,2047.18,1.77131,1.49563,1023.55,1.88262,1.57206,387.571,494,2,mo_BGO40_17_14_3_Family_129f_Bond_MP97_05_0001.sfrm,2024-07-19T09:11:30,89.4,34.7796,4.0,1800.0,14,17,-3,390.3,503.748,403.448,503.903,4329,442.888,4232.83,1.6236,1.54672,2276.27,1.68874,1.52489,1,mo_BGO40_17_14_3_Family_129f_Bond_MP97_02_0001.sfrm,2024-07-19T08:40:28,-90.6,4.0,1800.0,389.91,504.614,402.953,504.69,4574,406.897,4222.58,1.56374,1.4083,2254.61,1.67666,1.49195,390.105,97.0764,-0.00604661,10.5187,0.000490357


In [20]:
XLSX.openxlsx("$(name).xlsx", mode="w") do xf
    sheet = xf[1]
    XLSX.rename!(sheet, "$(name)")
    XLSX.writetable!(xf[1], data)
    sheet = XLSX.addsheet!(xf, "bond")
    XLSX.writetable!(sheet, bond)
    sheet = XLSX.addsheet!(xf, "frid")
    XLSX.writetable!(sheet, frid)
    sheet = XLSX.addsheet!(xf, "eccentr")
    XLSX.writetable!(sheet, eccentr)
end